# Data Science | Final Project | Group 02

## Identify epigenetic alterations associated with Alzheimer’s disease and classification of gene expressions between healthy and sick patients

#### Carmen Calle Huerta, Christina Kirschbaum, Pushpa Koirala, Melika Moradi

## Our Project

Alzheimer’s disease is the most prevalent kind of dementia and a fatal brain ailment. More study into this illness might lead to a better understanding of the condition and more effective treatment options. 

In this project, ChIP-seq data for H3K27ac, H3K9ac, H3K122ac and H3K4me1 as well as RNA-seq data from the from the lateral temporal lobe of the human brain for young healthy patients, old heathy patients and patients with Alzheimers disease will be analyzed and the differences between normal aging and cognitive impairment will be explored. The epigenomic or transcriptomic profiles will be analyzed to find relevant epigenetic alterations associated with the disease and help to better understand the molecular pathophysiology underlying.

Afterwards, with Machine Learning models the presence and absence of Alzheimer’s disease based on the data. The models will be built with Support Vector Machines and Random Forest.

Finally, the findings will be compared to them in related papers, we will look into relevant *in vivo* experiments with model organisms and use the Genome Browser to generate ChIP-seq tracks.

For our project we were inspirated by the paper:
Nativio R, Lan Y, Donahue G et al. ["An integrated multi-omics approach identifies
epigenetic alterations associated with Alzheimer’s disease."](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8098004/)

The data is derived from GEO, [Series GSE153875](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE153875), for the CHiP-seq data and the raw RNA-seq data [Series GSE159699](https://www.ncbi.nlm.nih.gov/Traces/study/?acc=PRJNA670209&o=acc_s%3Aa). 

## Import packages

First, we import some of the most frequently used packages in Python. [NumPy](https://numpy.org/doc/stable/) for working with arrays, matrices and linear algebra, [pandas](https://pandas.pydata.org/docs/) for data analysis and manipulation, [matplotlib](https://matplotlib.org/) and [seaborn](https://seaborn.pydata.org/) for visualizations.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data Integration and Preprocessing RNA-seq

In the next step, we import our RNA-seq data from the SubSeries GSE159699 of the SuperSeries GSE153875 with SRA-Toolkit and fastq-dump. To integrate this process into Python, the package subrocess is used.

In [2]:
import subprocess

# sra numbers from accession list of our RNA-seq data from SuperSeries GSE153875 / SubSeries GSE159699
sra_numbers = [
    "SRR12850830", "SRR12850831", "SRR12850832", "SRR12850833", "SRR12850834",
    "SRR12850835", "SRR12850836", "SRR12850837", "SRR12850838", "SRR12850839",
    "SRR12850840", "SRR12850841", "SRR12850842", "SRR12850843", "SRR12850844",
    "SRR12850845", "SRR12850846", "SRR12850847", "SRR12850848", "SRR12850849",
    "SRR12850850", "SRR12850851", "SRR12850852", "SRR12850853", "SRR12850854",
    "SRR12850855", "SRR12850856", "SRR12850857", "SRR12850858", "SRR12850859"
    ]

In [ ]:
# code from https://erilu.github.io/python-fastq-downloader/

# this will download the .sra files to ~/ncbi/public/sra/ 
for sra_id in sra_numbers:
    print ("Currently downloading: " + sra_id)
    prefetch = "prefetch " + sra_id
    print ("The command used was: " + prefetch)
    subprocess.call(prefetch, shell=True)

# this will extract the .sra files from above into a folder named 'fastq'
for sra_id in sra_numbers:
    print ("Generating fastq for: " + sra_id)
    fastq_dump = "fastq-dump --outdir fastq --gzip --skip-technical  --readids --read-filter pass --dumpbase --split-3 --clip ~/ncbi/public/sra/" + sra_id + ".sra"
    print ("The command used was: " + fastq_dump)
    subprocess.call(fastq_dump, shell=True)

## Data Integration for ChIP-seq

The .bed and .bw files for the ChIP-seq data were downloaded. They are available as supplementary files of the SuperSeries GSE153875 from GEO. We split them into folders for H3K27ac, H3K9ac, H3K122ac and H3K4me1 (and peaks) to get smaller sets.

In [ ]:
# The following code can get the metadata (if needed) for the ChIP-seq data with the library GEOparse (similar to GEOquery).
import GEOparse

gse = GEOparse.get_GEO(geo="GSE153875", destdir="./")

In [ ]:
# prints the metadata for the first sample
for gsm_name, gsm in gse.gsms.items():
    print("Name: ", gsm_name)
    print("Metadata:",)
    for key, value in gsm.metadata.items():
        print(" - %s : %s" % (key, ", ".join(value)))
    break

## Quality Control RNA-seq

### FastQC

For the Quality Control of the RNA-seq data, [FastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/) was downloaded and reports were generated for every sequence. The reports can be found in the repository in the folder MultiQC_RNA.

In [58]:
import subprocess

path = "/mnt/c/Users/Christina/Documents/GitHub/DataScience_finalProjekt_Group2/data/"
path_genomeDir = "/mnt/c/Users/Christina/Documents/GitHub/DataScience_finalProjekt_Group2/GenomeDir/"
path_fastq = "/mnt/c/Users/Christina/Documents/GitHub/DataScience_finalProjekt_Group2/data/fastq/"

In [6]:
# unzip data
for fastq in sra_numbers:
    gunzip = "gunzip " + path_fastq + fastq + "_pass.fastq.gz"
    subprocess.call(gunzip, shell=True)

In [ ]:
for fastq in sra_numbers:
    fastqc = "/home/christina/FastQC/fastqc \
              --readFilesIn " + path_fastq + fastq + "_pass.fastq"
    subprocess.call(fastqc, shell=True)

### STAR

Afterwards, we will now preprocess the data with STAR, an aligner for RNA-seq data mapping. Like in the paper of *Nativio et al.*, we will align our RNA-seq reads to the human reference genome (assembly GRCh37.75/hg19) using [STAR](https://github.com/alexdobin/STAR) with default parameters.
The [fasta](http://ftp.ensembl.org/pub/release-75/fasta/homo_sapiens/dna/) and the [gtf](http://ftp.ensembl.org/pub/release-75/gtf/homo_sapiens/) file used for the genomeGenerate run were downloaded from Ensembl. 

This will create .sam files out of the .fastq files. The logfiles for all sequences are saved in the folder MultiQC_RNA.

In [4]:
# get Genome Index (assembly GRCh37.75)
genomeIndex = "STAR --runThreadN 2 \
               --runMode genomeGenerate \
               --genomeDir " + path_genomeDir + " \
               --genomeFastaFiles " + path + "Homo_sapiens.GRCh37.75.dna_sm.primary_assembly.fa \
               --sjdbGTFfile " + path + "Homo_sapiens.GRCh37.75.gtf"
subprocess.call(genomeIndex, shell=True)

134

The logfile for genomeGenerate included many warnings for assembly GRCh37.75/h19, so the most recent version available was taken from Ensembl to check. The [fasta](http://ftp.ensembl.org/pub/release-106/fasta/homo_sapiens/dna/) (primary assembly soft masked) and the [gtf](http://ftp.ensembl.org/pub/release-106/gtf/homo_sapiens/) file can be found following the links.

In the check with the most recent version of GRCh38, the warnings were absent. 
The problem was that the patches, which suggest how gaps in the genome sequence can be filled, are not integrated in the version GRCh37.75/h19, but in the most recent version of GRCh38.

In [10]:
# get Genome Index (most recent)
genomeIndex = "STAR --runThreadN 2 \
               --runMode genomeGenerate \
               --genomeDir " + path_genomeDir + " \
               --genomeFastaFiles " + path + "Homo_sapiens.GRCh38.dna_sm.primary_assembly.fa \
               --sjdbGTFfile " + path + "Homo_sapiens.GRCh38.106.gtf"
subprocess.call(genomeIndex, shell=True)

134

To get rid of the warnings, the patches were removed from the assembly and the Genome Index was built with the new file. Due to the removed patches the read count can be distorted because reads that should align to the patches align somewhere else.

The argument `genomeSAindexNbases` was added and set to 10 according to the formula min(14, log2(GenomeLength)/2-1). The value is 14 by default and typically between 10 and 15. Longer strings use more memory but allow faster searches.

In [7]:
# unzip files without patches
gunzip = "gunzip " + path + "/without_patches/Homo_sapiens.GRCh37.75.dna_sm.*.fa.gz"
subprocess.call(gunzip, shell=True)

0

In [59]:
genomeIndex = "STAR --runThreadN 2 \
               --runMode genomeGenerate \
               --genomeDir " + path_genomeDir + " \
               --genomeFastaFiles " + path + "Homo_sapiens.GRCh37.75.dna_sm.primary_assembly.fa \
               --sjdbGTFfile " + path + "Homo_sapiens.GRCh37.75.without_patch.gtf \
               --genomeSAindexNbases 10"
subprocess.call(genomeIndex, shell=True)

137

In [6]:
# Run mapping
for fastq in sra_numbers:
    print ("Currently processing: " + fastq)
    mapping = "STAR --runThreadN 2 \
               --genomeDir " + path_genomeDir + " \
               --readFilesIn " + path_fastq + fastq + "_pass.fastq \
               --outFileNamePrefix " + path + fastq + "/"
    subprocess.call(mapping, shell=True)

Currently processing: SRR12850830
Currently processing: SRR12850831
Currently processing: SRR12850832
Currently processing: SRR12850833
Currently processing: SRR12850834
Currently processing: SRR12850835
Currently processing: SRR12850836
Currently processing: SRR12850837
Currently processing: SRR12850838
Currently processing: SRR12850839
Currently processing: SRR12850840
Currently processing: SRR12850841
Currently processing: SRR12850842
Currently processing: SRR12850843
Currently processing: SRR12850844
Currently processing: SRR12850845
Currently processing: SRR12850846
Currently processing: SRR12850847
Currently processing: SRR12850848
Currently processing: SRR12850849
Currently processing: SRR12850850
Currently processing: SRR12850851
Currently processing: SRR12850852
Currently processing: SRR12850853
Currently processing: SRR12850854
Currently processing: SRR12850855
Currently processing: SRR12850856
Currently processing: SRR12850857
Currently processing: SRR12850858
Currently proc

### featureCounts

Finally, they used featureCounts in the paper. featureCounts is part of the [Subread/RSubread](http://subread.sourceforge.net/) package.

In [ ]:
for sam in sra_numbers:
    print ("Currently processing: " + sam)
    featurecounts = "featureCounts -T 2 -t exon -g gene_id \
                     -a Homo_sapiens.GRCh37.75.gtf \
                     -o " + path + "/MultiQC_RNA/" + sam + ".txt \
                     Aligned_" + sam + ".out.sam"
    subprocess.call(featurecounts, shell=True)

### MultiQC

Now all the generated reports and logfiles for the RNA-seq will be summarized by the tool [MultiQC](https://multiqc.info/).

In [11]:
#multiqc = "multiqc /mnt/c/Users/Christina/Documents/GitHub/DataScience_finalProjekt_Group2/MultiQC_RNA/"
#subprocess.call(multiqc, shell=True)

import multiqc
multiqc.run("/mnt/c/Users/Christina/Documents/GitHub/DataScience_finalProjekt_Group2/MultiQC_RNA/")

0

## Quality Control ChIP-seq

Script for bw to fastq, Melika or Pushpa please try

Change paths to your directory:

path = where your hg19.fa is, can be deleted when it is in the directory you run it in

path_bw = path to the folder with the bw files

In [53]:
import subprocess

path = "/mnt/c/Users/Christina/Documents/GitHub/DataScience_finalProjekt_Group2/data/"
path_bw = "/mnt/c/Users/Christina/Documents/GitHub/DataScience_finalProjekt_Group2/data/H3K122ac/"

In [54]:
import os

files = os.listdir(path_bw)
for file in files:
    if file.endswith('.gz'):
        files.remove(file) 
files = [x[:-3] if x.endswith('.bw') else x for x in files]

In [57]:
# for .bw files

for file in files:
    bigwigtowig = "bigWigToWig " + path_bw + file + ".bw " + path_bw + file +".wig"
    subprocess.call(bigwigtowig, shell=True)

    wigtobed = "wig2bed < " + path_bw + file + ".wig > " + path_bw + file + ".bed"
    subprocess.call(wigtobed, shell=True)

    getfasta = "bedtools getfasta -fi " + path + "hg19.fa -bed " + path_bw + file + ".bw -fo " + path_bw + file + ".fa"
    subprocess.call(getfasta, shell=True)

    perl = "perl " + path + "fasta_to_fastq.pl" + path_bw + file + ".fa > " + path_bw + file + ".fq"
    subprocess.call(perl, shell=True)

## Exploratory Data Analysis